In [ ]:
# TODO UPDATE THIS OLD MESSY SCRIPT

In [15]:
# IMPORT PACKAGES
import pandas as pd

In [16]:
# READ IN THE DATA

cmc_df       = pd.read_pickle('../3-data/derived/cmc_cw.pkl')
coinapi_df   = pd.read_pickle('../3-data/derived/coinapi_cw.pkl')
coingecko_df = pd.read_pickle('../3-data/derived/coingecko_cw.pkl')
cm_df        = pd.read_pickle('../3-data/derived/coinmetrics_cw.pkl')
messari_df   = pd.read_pickle('../3-data/derived/messari_cw.pkl')
san_df       = pd.read_pickle('../3-data/derived/santiment_cw.pkl')


In [17]:
# MERGE DATA TOGETHER

df = cmc_df.merge(coinapi_df,
                  on='cmc_id',
                  how='outer',
                  validate='one_to_one')
assert(0==df[df.symbol_cmc.isnull()].shape[0]) # note: ensure no cmc_id's in non cmc df that aren't in cmc df

for merging_df in [coingecko_df, cm_df, messari_df, san_df]:
    df = df.merge(merging_df,
                  on='cmc_id',
                  how='outer',
                  validate='one_to_one')
    assert(0==df[df.symbol_cmc.isnull()].shape[0]) # note: ensure no cmc_id's in non cmc df that aren't in cmc df


In [18]:
# Drop column that is also in the santiment panel
df = df.drop('total_supply_san', axis=1)


In [19]:
# CONFIRM ALL ID COLUMNS ARE UNIQUE

assert(df[~df.cmc_id.isnull()].cmc_id.is_unique)
assert(df[~df.coinapi_id.isnull()].coinapi_id.is_unique)
assert(df[~df.gecko_id.isnull()].gecko_id.is_unique)
assert(df[~df.cm_id.isnull()].cm_id.is_unique)
assert(df[~df.messari_id.isnull()].messari_id.is_unique)
assert(df[~df.san_id.isnull()].san_id.is_unique)


In [20]:
# SORT 

# Reorder cols
cols = list(df.columns.values)
id_cols = ['cmc_id', 'coinapi_id', 'gecko_id',
           'cm_id', 'messari_id', 'san_id']
for col in id_cols:
    cols.remove(col)
df = df[id_cols + cols]

# Sort rows
df = df.sort_values(by=['cmc_id'], ignore_index=True)


In [21]:
# OUTPUT
df.to_pickle('../3-data/clean/cw.pkl')
